In [29]:
import numpy as np
import string
import pandas as pd
from io import StringIO
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, LSTM, Dropout, Bidirectional, Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

MAX_WORDS = 15000

PREPARE LABELS

In [30]:
df = pd.read_json("trump_tweets_11_17.json")
replacement_dict = {"Twitter for iPhone": 1, "Twitter for Android": 0}
df = df.replace(to_replace=replacement_dict)   # replace labels
df = df[df.source.apply(lambda x: type(x) == int)]  # remove tweets from other sources
df = df.sample(frac=1)

Tokenize text

In [31]:
training = df.sample(frac=0.8, random_state=0)
testing = df.drop(training.index)

Split into training and testing

In [36]:
tokenizer = Tokenizer(num_words=15000)
tokenizer.fit_on_texts(training["text"])
sequences = tokenizer.texts_to_sequences(training["text"])
text_sequences = pad_sequences(sequences, maxlen=65)
dictionary = tokenizer.word_index

In [45]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

model = Sequential()
model.add(Dense(512, input_shape=(65,), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='softmax'))

In [48]:
model.compile(loss='mse',
  optimizer='adam',
  metrics=['accuracy'])

In [49]:
model.fit(text_sequences, training["source"],
  batch_size=32,
  epochs=5,
  verbose=1,
  validation_split=0.1,
  shuffle=True)


Train on 13391 samples, validate on 1488 samples
Epoch 1/5
13391/13391 [==============================] - 4s 326us/step - loss: 0.2378 - accuracy: 0.7622 - val_loss: 0.2392 - val_accuracy: 0.7608
Epoch 2/5
13391/13391 [==============================] - 3s 253us/step - loss: 0.2378 - accuracy: 0.7622 - val_loss: 0.2392 - val_accuracy: 0.7608
Epoch 3/5
13391/13391 [==============================] - 3s 245us/step - loss: 0.2378 - accuracy: 0.7622 - val_loss: 0.2392 - val_accuracy: 0.7608
Epoch 4/5
13391/13391 [==============================] - 3s 222us/step - loss: 0.2378 - accuracy: 0.7622 - val_loss: 0.2392 - val_accuracy: 0.7608
Epoch 5/5
13391/13391 [==============================] - 3s 236us/step - loss: 0.2378 - accuracy: 0.7622 - val_loss: 0.2392 - val_accuracy: 0.7608


In [44]:
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 512)               33792     
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_8 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 2)                 514       
Total params: 165,634
Trainable params: 165,634
Non-trainable params: 0
_________________________________________________________________
None
